In [1]:
import numpy as np
import plotly.graph_objects as go
from ipywidgets import interactive, HBox, VBox, FloatSlider

# Define the function to update the plot based on the slider values
def update_plot(Dose, Vd, CL):
    
    # Calculate derived variables
    C0 = Dose / Vd
    ke = CL / Vd
    t_half = np.log(2) / ke

    # Initialize arrays to store the results over time
    time_steps = np.arange(0, 102, 1)  # time from 0 to 101
    B = np.zeros_like(time_steps, dtype=float)  # amount of drug
    C = np.zeros_like(time_steps, dtype=float)  # concentration of drug
    D = np.zeros_like(time_steps, dtype=float)  # ln(concentration of drug)
    E = np.zeros_like(time_steps, dtype=float)  # cumulative AUC

    # Set initial values
    B[0] = Dose
    C[0] = C0

    # Loop through each time step to calculate the values based on the formulas
    for t in range(1, len(time_steps)):
        B[t] = B[t-1] - CL * C[t-1]
        C[t] = B[t] / Vd
        D[t] = np.log(C[t])
        E[t] = E[t-1] + 0.5 * (C[t-1] + C[t])

    # Create the plot using Plotly
    fig = go.Figure()
    
    # Add drug concentration over time plot
    fig.add_trace(go.Scatter(x=time_steps, y=C, mode='lines', name='Drug concentration over time'))
    
    # Add natural log of drug concentration over time plot
    fig.add_trace(go.Scatter(x=time_steps, y=D, mode='lines', name='Natural log of drug concentration over time', line=dict(color='red')))

    # Update layout to create two rows and titles
    fig.update_layout(
        title="Pharmacokinetics of the Drug: Single Intravenous Dose Administration",
        template="plotly_dark",
        xaxis_title="Time (min)",
        yaxis_title="Drug Concentration (mg/l)",
        updatemenus=[
            dict(
                buttons=list([
                    dict(
                        args=[{'yaxis.domain': [0.55, 1]}],
                        label='Drug Concentration over Time',
                        method='relayout'
                    ),
                    dict(
                        args=[{'yaxis.domain': [0, 0.45]}],
                        label='Natural Log of Drug Concentration over Time',
                        method='relayout'
                    )
                ]),
                direction='down',
                pad={'r': 10, 't': 10},
                showactive=True,
                x=0.17,
                xanchor='left',
                y=1.15,
                yanchor='top'
            ),
        ]
    )

    # Update the size of the figure
    fig.update_layout(height=600, width=800)

    # Display the plot
    fig.show()

# Create sliders
Dose_slider = FloatSlider(min=0, max=200, step=1, value=100, description='Dose (mg)')
Vd_slider = FloatSlider(min=0, max=100, step=1, value=40, description='Vd (l)')
CL_slider = FloatSlider(min=0, max=10, step=0.1, value=3, description='CL (l/min)')

# Create interactive widget
widget = interactive(update_plot, Dose=Dose_slider, Vd=Vd_slider, CL=CL_slider)

# Display the sliders and the plot
output = widget.children[-1]
output.layout.height = '600px'
VBox([HBox(widget.children[0:3]), output])